# Requirements

In [1]:
import qpsolvers
from os.path import exists
import pickle

import sn_bayes
from sn_bayes import longevity_bayes
from sn_bayes.utils import complexity_check
from sn_bayes.utils import get_var_positions
from sn_bayes.utils import get_var_val_positions
from sn_bayes.utils import make_tree
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import query
from sn_bayes.utils import internal_query
from sn_bayes.utils import internal_query
from sn_bayes.utils import explain_why_bad
from sn_bayes.utils import explain_why_good
from sn_bayes.utils import create_query
from sn_bayes.utils import dependency
from sn_bayes.utils import addCpt
from sn_bayes.utils import non_cpt_descriptions
from sn_bayes.utils import any_of
from sn_bayes.utils import all_of
from sn_bayes.utils import avg
from sn_bayes.utils import if_then_else
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import non_cpt_descriptions

from sn_service.service_spec.bayesian_pb2 import BayesianNetwork

import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import Query
import sn_service.service_spec.bayesian_pb2_grpc as grpc_bayes_grpc
import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetworkQuery
from sn_service.service_spec.bayesian_pb2 import QueryId
from sn_service.service_spec.bayesian_pb2 import Id

import grpc
import pandas as pd
import networkx as nx
import time
import re

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

C:\Users\Osvaldo\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Osvaldo\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Example 1

In [2]:
bayesianNetwork = BayesianNetwork()

In [3]:
cpt = {}
outstr = ''
# Define a Bayesian Network structure with discrete distributions for hypertension
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "hypertension"
variable = discreteDistribution.variables.add()
variable.name = "hypertension"
variable.probability = 0.50
variable = discreteDistribution.variables.add()
variable.name = "no_hypertension"
variable.probability = 0.50
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "bmi"
variable = discreteDistribution.variables.add()
variable.name = "bmi_over_40_high_risk"
variable.probability = 0.08
variable = discreteDistribution.variables.add()
variable.name = "bmi_35_to_39_moderate_risk"
variable.probability = 0.10
variable = discreteDistribution.variables.add()
variable.name = "bmi_30_to_34_low_risk"
variable.probability = 0.20
variable = discreteDistribution.variables.add()
variable.name = "bmi_25_to_29_overweight"
variable.probability = 0.30
variable = discreteDistribution.variables.add()
variable.name = "bmi_under_25_healthy"
variable.probability = 0.32
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "age"
variable = discreteDistribution.variables.add()
variable.name = "elderly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "adult"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "young_adult"
variable.probability = 0.3
variable = discreteDistribution.variables.add()
variable.name = "teen"
variable.probability = 0.2
variable = discreteDistribution.variables.add()
variable.name = "child"
variable.probability = 0.2
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "dietary_folate"
variable = discreteDistribution.variables.add()
variable.name = "dietary_folate_119.00_and_below"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "dietary_folate_above_119.00_to_264.00_and_below"
variable.probability = 0.20
variable = discreteDistribution.variables.add()
variable.name = "dietary_folate_above_264.00_to_410.00_and_below"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "dietary_folate_above_410.00_to_622.00_and_below"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "dietary_folate_above_622.00"
variable.probability = 0.25
# Define CPT entries
cpt["neither_hypertension_nor_bmi_over_40"] = all_of(bayesianNetwork, cpt,
{
    'hypertension': {"no_hypertension"},
    'bmi': {"bmi_35_to_39_moderate_risk", "bmi_30_to_34_low_risk", "bmi_25_to_29_overweight", "bmi_under_25_healthy"}
},
["neither_hypertension_nor_bmi_over_40", "not_neither_hypertension_nor_bmi_over_40"]
)
# Define CPT entries
cpt["both_hypertension_and_bmi_over_40"] = all_of(bayesianNetwork, cpt,
{
    'hypertension': {"hypertension"},
    'bmi': {"bmi_over_40_high_risk"}
},
["both_hypertension_and_bmi_over_40", "not_both_hypertension_and_bmi_over_40"]
)
# Update the CPT in the Bayesian Network - Calculate dependencies to update CPT based on relative risk adjustments
outstr = outstr + addCpt(bayesianNetwork, cpt)
cpt = {}
cpt["disorders_of_lipid_metabolism"] = dependency(bayesianNetwork, cpt,
[
    ({"both_hypertension_and_bmi_over_40": ["both_hypertension_and_bmi_over_40"]}, {"relative_risk": 2}),
    ({"neither_hypertension_nor_bmi_over_40": ["neither_hypertension_nor_bmi_over_40"]}, {"relative_risk": 2}),
],
{"disorders_of_lipid_metabolism":0.2175, "no_disorders_of_lipid_metabolism":0.7825},
# adjust=True
)
# Update the CPT in the Bayesian Network
outstr = outstr + addCpt(bayesianNetwork, cpt)
cpt = {}
cpt["dna_methylation"] = dependency(bayesianNetwork,cpt, 
[
({"dietary_folate":["dietary_folate_above_622.00"]},{"relative_risk":0.9}),
({"age":["adult"]},{"relative_risk":5}),
({"age":["elderly"]},{"relative_risk":10})
],
{"dna_methylation_above_.3":0.38,"dna_methylation_above_.3_no":0.62}
)
#
longevity = bayesInitialize(bayesianNetwork)
longevity.bake()
# Calls a Pomegranate routine that shows the computed probabilities of every variable. 
longevity.predict_proba({}) # We have our own utility that pulls out particular variables.
# First example query
evidence = {"bmi": "bmi_over_40_high_risk", "hypertension": "hypertension"}
outvars = ["disorders_of_lipid_metabolism"]
results = query(longevity, bayesianNetwork, evidence, outvars)
results

start timing...
start timing...
[({'both_hypertension_and_bmi_over_40': ['both_hypertension_and_bmi_over_40']}, {'relative_risk': 2}), ({'neither_hypertension_nor_bmi_over_40': ['neither_hypertension_nor_bmi_over_40']}, {'relative_risk': 2})] ==> {'disorders_of_lipid_metabolism': 0.2175, 'no_disorders_of_lipid_metabolism': 0.7825} took 0.01837080000000002 seconds
{'disorders_of_lipid_metabolism': 0.2175, 'no_disorders_of_lipid_metabolism': 0.7825}  wrapper took 0.1018110000000001 seconds
start timing...
start timing...
[({'dietary_folate': ['dietary_folate_above_622.00']}, {'relative_risk': 0.9}), ({'age': ['adult']}, {'relative_risk': 5}), ({'age': ['elderly']}, {'relative_risk': 10})] ==> {'dna_methylation_above_.3': 0.38, 'dna_methylation_above_.3_no': 0.62} took 0.0262289 seconds
{'dna_methylation_above_.3': 0.38, 'dna_methylation_above_.3_no': 0.62}  wrapper took 0.02630050000000006 seconds


{'disorders_of_lipid_metabolism': {'disorders_of_lipid_metabolism': 0.4182692766189576,
  'no_disorders_of_lipid_metabolism': 0.5817307233810424}}

In [4]:
bayesianNetwork = BayesianNetwork()

cpt = {}
outstr = ''
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "bmi"
variable = discreteDistribution.variables.add()
variable.name = "bmi_over_40_high_risk"
variable.probability = 0.08
variable = discreteDistribution.variables.add()
variable.name = "bmi_35_to_39_moderate_risk"
variable.probability = 0.10
variable = discreteDistribution.variables.add()
variable.name = "bmi_30_to_34_low_risk"
variable.probability = 0.20
variable = discreteDistribution.variables.add()
variable.name = "bmi_25_to_29_overweight"
variable.probability = 0.30
variable = discreteDistribution.variables.add()
variable.name = "bmi_under_25_healthy"
variable.probability = 0.32
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "dietary_energy"
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_3939.00"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_2612.00_to_3939.00_and_below"
variable.probability = 0.20
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_1930.00_to_2612.00_and_below"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_above_1399.00_to_1930.00_and_below"
variable.probability = 0.25
variable = discreteDistribution.variables.add()
variable.name = "dietary_energy_1399.00_and_below"
variable.probability = 0.25
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "shortness_of_breath_exertion"
variable = discreteDistribution.variables.add()
variable.name = "shortness_of_breath_exertion_yes"
variable.probability = 0.38
variable = discreteDistribution.variables.add()
variable.name = "shortness_of_breath_exertion_no"
variable.probability = 0.62
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "daily_walk_or_bicycle_travel"
variable = discreteDistribution.variables.add()
variable.name = "daily_walk_or_bicycle_travel_less_than_30_minutes"
variable.probability = 0.34
variable = discreteDistribution.variables.add()
variable.name = "daily_walk_or_bicycle_travel_30_to_59_minutes"
variable.probability = 0.30
variable = discreteDistribution.variables.add()
variable.name = "daily_walk_or_bicycle_travel_60_minutes_or_more"
variable.probability = 0.36
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "daily_time_sitting"
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_12_or_more_hours"
variable.probability = 0.08
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_over_6_but_less_than_12_hours"
variable.probability = 0.36
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_over_2_but_less_than_6_hours"
variable.probability = 0.49
variable = discreteDistribution.variables.add()
variable.name = "daily_time_sitting_less_than_2_hours"
variable.probability = 0.06
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "physical_work"
variable = discreteDistribution.variables.add()
variable.name = "physical_work_no"
variable.probability = 0.76
variable = discreteDistribution.variables.add()
variable.name = "physical_work_yes"
variable.probability = 0.24
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "ten_minutes_daily_workout"
variable = discreteDistribution.variables.add()
variable.name = "ten_minutes_daily_workout_no"
variable.probability = 0.60
variable = discreteDistribution.variables.add()
variable.name = "ten_minutes_daily_workout_yes"
variable.probability = 0.40
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "steps_anomaly"
variable = discreteDistribution.variables.add()
variable.name = "steps_anomaly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_steps_anomaly"
variable.probability = 0.95
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "workout_anomaly"
variable = discreteDistribution.variables.add()
variable.name = "workout_anomaly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_workout_anomaly"
variable.probability = 0.95
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "walking_speed_anomaly"
variable = discreteDistribution.variables.add()
variable.name = "walking_speed_anomaly"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_walking_speed_anomaly"
variable.probability = 0.95
#
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "cumin_supplementation"
variable = discreteDistribution.variables.add()
variable.name = "cumin_supplementation_yes"
variable.probability = 0.01
variable = discreteDistribution.variables.add()
variable.name = "cumin_supplementation_no"
variable.probability = 0.99
#
cpt["lack_of_workout_reported"] = all_of(bayesianNetwork,cpt,
        {
            "daily_walk_or_bicycle_travel":"daily_walk_or_bicycle_travel_less_than_30_minutes",
            "physical_work":"physical_work_no",
            'ten_minutes_daily_workout':"ten_minutes_daily_workout_no"
                },
        ["lack_of_workout_reported","no_lack_of_workout_reported"]
        )
#
cpt["lack_of_exercise_reported"] = any_of(bayesianNetwork,cpt,
        {
            "shortness_of_breath_exertion":"shortness_of_breath_exertion_yes",
            "daily_time_sitting":"daily_time_sitting_12_or_more_hours",
            "lack_of_workout_reported":"lack_of_workout_reported"
                },
        ["lack_of_exercise_reported","no_lack_of_exercise_reported"]

        )
#
cpt["lack_of_exercise_signal"] = any_of(bayesianNetwork,cpt,
        {
 "workout_anomaly":{ "workout_anomaly"},
 "walking_speed_anomaly":{"walking_speed_anomaly"},
 "steps_anomaly":{"steps_anomaly"}
 },
 ["lack_of_exercise_signal","no_lack_of_exercise_signal"]
 )
#
# cpt["lack_of_exercise_signal"] = avg(bayesianNetwork,cpt,
#         [
#  "workout_anomaly",
#  "walking_speed_anomaly",
#  "steps_anomaly"
#  ],
#  ["lack_of_exercise_signal","no_lack_of_exercise_signal"]
#  )
#
cpt["lack_of_exercise"] = avg(bayesianNetwork,cpt,
 [
 "lack_of_exercise_signal",
 "lack_of_exercise_reported",
 ],
 ["lack_of_exercise","no_lack_of_exercise"]
 )
#
outstr = outstr + addCpt(bayesianNetwork,cpt) 
cpt = {}
cpt["workday_sleep_hours_per_night"] = dependency(bayesianNetwork,cpt, 
[
({"ten_minutes_daily_workout":["ten_minutes_daily_workout_no"]},{"relative_risk":2.341285178, "plus_minus": 0.866177854, "ci": 95})
],
{"workday_sleep_under_5":0.06,"workday_sleep_under_5_no":0.94}
)
#
outstr = outstr + addCpt(bayesianNetwork,cpt) 
cpt = {}
cpt["obesity_factors"] = dependency(bayesianNetwork,cpt, 
[
({"daily_time_sitting":["daily_time_sitting_12_or_more_hours"]},{"relative_risk":2.5, "plus_minus": 0.15, "ci": 95}),
({"workday_sleep_hours_per_night":["workday_sleep_under_5"]},{"relative_risk":1.38, "plus_minus": 0.13, "ci": 95}),
({"cumin_supplementation":["cumin_supplementation_yes"]},{"relative_risk":0.2973932327, "plus_minus": 0.2143891494, "ci": 95}),
({"dietary_energy":["dietary_energy_above_3939.00"]},{"sensitivity":0.1, "specificity":0.95, "plus_minus": 0.01, "ci": 95}),
({"dietary_energy":["dietary_energy_above_2612.00_to_3939.00_and_below"]},{"sensitivity":0.3, "specificity":0.7, "plus_minus": 0.01, "ci": 95}),
({"lack_of_exercise":["lack_of_exercise"]},{"sensitivity":0.4, "specificity":0.8})
],
{"obesity_factors":0.38,"no_obesity_factors":0.62}
)
#
outstr = outstr + addCpt(bayesianNetwork,cpt) 
cpt = {}
cpt["obesity"] = dependency(bayesianNetwork,cpt, 
[
({"bmi":["bmi_over_40_high_risk"]},{"sensitivity":0.21, "specificity":0.99, "plus_minus": 0.01, "ci": 95}),
({"bmi":["bmi_35_to_39_moderate_risk"]},{"sensitivity":0.27, "specificity":0.99, "plus_minus": 0.01, "ci": 95}),
({"obesity_factors":["obesity_factors"]},{"sensitivity":0.95, "specificity":0.95, "plus_minus": 0.1, "ci": 95})
],
{"obesity":0.38,"no_obesity":0.62}
)
#
longevity = bayesInitialize(bayesianNetwork)
longevity.bake()
# Calls a Pomegranate routine that shows the computed probabilities of every variable. 
longevity.predict_proba({}) # We have our own utility that pulls out particular variables.
#
evidence = {"physical_work":"physical_work_no",'ten_minutes_daily_workout':"ten_minutes_daily_workout_no","daily_time_sitting":"daily_time_sitting_12_or_more_hours",}
outvars = ["lack_of_exercise"]
results = query(longevity,bayesianNetwork,evidence,outvars)
results

start timing...
start timing...
[({'ten_minutes_daily_workout': ['ten_minutes_daily_workout_no']}, {'relative_risk': 2.341285178, 'plus_minus': 0.866177854, 'ci': 95})] ==> {'workday_sleep_under_5': 0.06, 'workday_sleep_under_5_no': 0.94} took 0.021333900000000128 seconds
{'workday_sleep_under_5': 0.06, 'workday_sleep_under_5_no': 0.94}  wrapper took 0.024530900000000244 seconds
start timing...
start timing...
[({'daily_time_sitting': ['daily_time_sitting_12_or_more_hours']}, {'relative_risk': 2.5, 'plus_minus': 0.15, 'ci': 95}), ({'workday_sleep_hours_per_night': ['workday_sleep_under_5']}, {'relative_risk': 1.38, 'plus_minus': 0.13, 'ci': 95}), ({'cumin_supplementation': ['cumin_supplementation_yes']}, {'relative_risk': 0.2973932327, 'plus_minus': 0.2143891494, 'ci': 95}), ({'dietary_energy': ['dietary_energy_above_3939.00']}, {'sensitivity': 0.1, 'specificity': 0.95, 'plus_minus': 0.01, 'ci': 95}), ({'dietary_energy': ['dietary_energy_above_2612.00_to_3939.00_and_below']}, {'sensiti

{'lack_of_exercise': {'lack_of_exercise': 0.5713125010410254,
  'no_lack_of_exercise': 0.4286874989589746}}